In [2]:
def comma_separated(xs):
    """Convert each value in the sequence xs to a string, and separate them
    with commas.

    >>> comma_separated(['spam', 5, False])
    'spam, 5, False'
    >>> comma_separated([5])
    '5'
    >>> comma_separated([])
    ''
    """
    return ', '.join([str(x) for x in xs])

In [3]:
import operator

class Expr:
    """
    When you type input into this interpreter, it is parsed (read) into an
    expression. This expression is represented in our code as an instance of
    this `Expr` class.

    In our interpreter, there are four types of expressions:
        - literals, which are simply numbers (e.g. 42 or 4.2)
        - names (e.g. my_awesome_variable_name)
        - call expressions (e.g. add(3, 4))
        - lambda expressions (e.g. lambda x: x)

    Call expressions and lambda expressions are built from simpler expressions.
    A lambda's body and a call expression's operator and operands are expressions
    as well. This means `Expr` is a recursive data structure, similar to a tree.
    This type of a tree is called an "abstract syntax tree".

    In our code, the four types of expressions are subclasses of the `Expr`
    class: `Literal`, `Name`, `CallExpr`, and `LambdaExpr`.
    """

    def __init__(self, *args):
        # The star (*) means that `args` will be a tuple of arguments passed to
        # this function.
        self.args = args

    def eval(self, env):
        """
        Each subclass of Expr implements its own eval method.

        `env` is a dictionary mapping strings to `Value` instances,
        representing the environment in which this expression is being
        evaluated.  This method should return a `Value` instance, the result of
        evaluating the expression.
        """
        raise NotImplementedError

    def __str__(self):
        """
        Returns a parsable and human-readable string of this expression (i.e.
        what you would type into the interpreter).

        >>> expr = CallExpr(LambdaExpr(['x'], Name('x')), [Literal(5)])
        >>> str(expr)
        '(lambda x: x)(5)'
        """
        raise NotImplementedError

    def __repr__(self):
        """
        Returns how this expression is written in our Python representation.

        >>> expr1 = LambdaExpr(['f'], CallExpr(Name('f'), [Literal(0)]))
        >>> expr1
        LambdaExpr(['f'], CallExpr(Name('f'), [Literal(0)]))

        >>> expr2 = CallExpr(LambdaExpr([], Literal(5)), [])
        >>> expr2
        CallExpr(LambdaExpr([], Literal(5)), [])
        """
        args = '(' + comma_separated([repr(arg) for arg in self.args]) + ')'
        return type(self).__name__ + args

class Literal(Expr):
    """A literal is notation for representing a fixed value in code. In
    PyCombinator, the only literals are numbers. A `Literal` should always
    evaluate to a `Number` value.

    The `value` attribute contains the fixed value the `Literal` refers to.
    """
    def __init__(self, value):
        Expr.__init__(self, value)
        self.value = value

    def eval(self, env):
        return Number(self.value)

    def __str__(self):
        return str(self.value)

class Name(Expr):
    """A `Name` is a variable. When evaluated, we look up the value of the
    variable in the current environment.

    The `string` attribute contains the name of the variable (as a Python
    string).
    """
    def __init__(self, string):
        Expr.__init__(self, string)
        self.string = string

    def eval(self, env):
        """
        >>> env = {
        ...     'a': Number(1),
        ...     'b': LambdaFunction([], Literal(0), {})
        ... }
        >>> Name('a').eval(env)
        Number(1)
        >>> Name('b').eval(env)
        LambdaFunction([], Literal(0), {})
        >>> try:
        ...     print(Name('c').eval(env))
        ... except NameError:
        ...     print('Exception raised!')
        Exception raised!
        """
        "*** YOUR CODE HERE ***"

        if self.string in list(env.keys()):
            return env[self.string]
        else:
            raise NameError('Name Error')        


    def __str__(self):
        return self.string

class LambdaExpr(Expr):
    """A lambda expression, which evaluates to a `LambdaFunction`.

    The `parameters` attribute is a list of variable names (a list of strings).
    The `body` attribute is an instance of `Expr`.

    For example, the lambda expression `lambda x, y: add(x, y)` is parsed as

    LambdaExpr(['x', 'y'], CallExpr(Name('add'), [Name('x'), Name('y')]))

    where `parameters` is the list ['x', 'y'] and `body` is the expression
    CallExpr('add', [Name('x'), Name('y')]).
    """
    def __init__(self, parameters, body):
        Expr.__init__(self, parameters, body)
        self.parameters = parameters
        self.body = body

    def eval(self, env):
        return LambdaFunction(self.parameters, self.body, env)

    def __str__(self):
        body = str(self.body)
        if not self.parameters:
            return 'lambda: ' + body
        else:
            return 'lambda ' + comma_separated(self.parameters) + ': ' + body

class CallExpr(Expr):
    """A call expression represents a function call.

    The `operator` attribute is an instance of `Expr`.
    The `operands` attribute is a list of `Expr` instances.

    For example, the call expression `add(3, 4)` is parsed as

    CallExpr(Name('add'), [Literal(3), Literal(4)])

    where `operator` is Name('add') and `operands` are [Literal(3), Literal(4)].
    """
    def __init__(self, operator, operands):
        Expr.__init__(self, operator, operands)
        self.operator = operator
        self.operands = operands

    def eval(self, env):
        """
        >>> from reader import read
        >>> new_env = global_env.copy()
        >>> new_env.update({'a': Number(1), 'b': Number(2)})
        >>> add = CallExpr(Name('add'), [Literal(3), Name('a')])
        >>> add.eval(new_env)
        Number(4)
        >>> new_env['a'] = Number(5)
        >>> add.eval(new_env)
        Number(8)
        >>> read('max(b, a, 4, -1)').eval(new_env)
        Number(5)
        >>> read('add(mul(3, 4), b)').eval(new_env)
        Number(14)
        """
        "*** YOUR CODE HERE ***"
        """
        return self.operator.string
        for i in self.operands:
            if(isinstance(i, Literal)):
                print(i.value)
            elif(isinstance(i, Name)):
                print(i.string)
            else:
                print(0)        
        """
        operator = self.operator.eval(env)
        operands = [operand.eval(env) for operand in self.operands]
        return operator.apply(operands)



    def __str__(self):
        function = str(self.operator)
        args = '(' + comma_separated(self.operands) + ')'
        if isinstance(self.operator, LambdaExpr):
            return '(' + function + ')' + args
        else:
            return function + args

class Value:
    """
    Values are the result of evaluating expressions. In an environment diagram,
    values appear on the right (either in a binding or off in the space to the
    right).

    In our interpreter, there are three types of values:
        - numbers (e.g. 42)
        - lambda functions, which are created by lambda expressions
        - primitive functions, which are functions that are built into the
            interpreter (e.g. add)

    In our code, the three types of values are subclasses of the `Value` class:
    Number, LambdaFunction, and PrimitiveFunction.
    """

    def __init__(self, *args):
        self.args = args

    def apply(self, arguments):
        """
        Each subclass of Value implements its own apply method.

        Note that only functions can be "applied"; attempting to apply a
        `Number` (e.g. as in 4(2, 3)) will raise an exception.

        For functions, `arguments` is a list of `Value` instances, the
        arguments to the function. It should return a `Value` instance, the
        result of applying the function to the arguments.
        """
        raise NotImplementedError

    def __str__(self):
        """
        Returns a parsable and human-readable version of this value (i.e. the
        output of this value to be displayed in the interpreter).
        """
        raise NotImplementedError

    def __repr__(self):
        """
        Returns how this value is written in our Python representation.
        """
        args = '(' + comma_separated([repr(arg) for arg in self.args]) + ')'
        return type(self).__name__ + args

class Number(Value):
    """A plain number. Attempting to apply a `Number` (e.g. as in 4(2, 3)) will
    raise an exception.

    The `value` attribute is the Python number that this represents.
    """
    def __init__(self, value):
        Value.__init__(self, value)
        self.value = value

    def apply(self, arguments):
        raise TypeError("Cannot apply number {} to arguments {}".format(
            self.value, comma_separated(arguments)))

    def __str__(self):
        return str(self.value)

In [6]:
class Buffer(object):
    """A Buffer provides a way of accessing a sequence one at a time.
    Its constructor takes a sequence, called the "source".
    The Buffer supplies elements from source one at a time through its remove_front()
    method. In addition, Buffer provides a current() method to look at the
    next item to be supplied, without moving past it.
    >>> buf = Buffer(['(', '+', 15, 12, ')'])
    >>> buf.remove_front()
    '('
    >>> buf.remove_front()
    '+'
    >>> buf.current()
    15
    >>> buf.remove_front()
    15
    >>> buf.current()
    12
    >>> buf.remove_front()
    12
    >>> buf.remove_front()
    ')'
    >>> buf.remove_front()  # returns None
    """
    def __init__(self, source):
        self.index = 0
        self.source = source

    def remove_front(self):
        """Remove the next item from self and return it. If self has
        exhausted its source, returns None."""
        current = self.current()
        self.index += 1
        return current

    def current(self):
        """Return the current element, or None if none exists."""
        if self.index >= len(self.source):
            return None
        else:
            return self.source[self.index]

    def expect(self, expected):
        actual = self.remove_front()
        if expected != actual:
            raise SyntaxError("expected '{}' but got '{}'".format(expected, actual))
        else:
            return actual

    def __str__(self):
        return str(self.source[self.index:])

In [7]:
import string


SYMBOL_STARTS = set(string.ascii_lowercase + string.ascii_uppercase + '_')
SYMBOL_INNERS = SYMBOL_STARTS | set(string.digits)
NUMERAL = set(string.digits + '-.')
WHITESPACE = set(' \t\n\r')
DELIMITERS = set('(),:')

def read(s):
    """Parse an expression from a string. If the string does not contain an
    expression, None is returned. If the string cannot be parsed, a SyntaxError
    is raised.

    >>> read('lambda f: f(0)')
    LambdaExpr(['f'], CallExpr(Name('f'), [Literal(0)]))
    >>> read('(lambda x: x)(5)')
    CallExpr(LambdaExpr(['x'], Name('x')), [Literal(5)])
    >>> read('(lambda: 5)()')
    CallExpr(LambdaExpr([], Literal(5)), [])
    >>> read('lambda x y: 10')
    Traceback (most recent call last):
      ...
    SyntaxError: expected ':' but got 'y'
    >>> read('  ')  # returns None
    """
    src = Buffer(tokenize(s))
    if src.current() is not None:
        return read_expr(src)

###########
## Lexer ##
###########
def tokenize(s):
    """Splits the string s into tokens and returns a list of them.

    >>> tokenize('lambda f: f(0, 4.2)')
    ['lambda', 'f', ':', 'f', '(', 0, ',', 4.2, ')']
    """
    src = Buffer(s)
    tokens = []
    while True:
        token = next_token(src)
        if token is None:
            return tokens
        tokens.append(token)

def take(src, allowed_characters):
    result = ''
    while src.current() in allowed_characters:
        result += src.remove_front()
    return result

def next_token(src):
    take(src, WHITESPACE)  # skip whitespace
    c = src.current()
    if c is None:
        return None
    elif c in NUMERAL:
        literal = take(src, NUMERAL)
        try:
            return int(literal)
        except ValueError:
            try:
                return float(literal)
            except ValueError:
                raise SyntaxError("'{}' is not a numeral".format(literal))
    elif c in SYMBOL_STARTS:
        return take(src, SYMBOL_INNERS)
    elif c in DELIMITERS:
        src.remove_front()
        return c
    else:
        raise SyntaxError("'{}' is not a token".format(c))

def is_literal(s):
    return isinstance(s, int) or isinstance(s, float)

def is_name(s):
    return isinstance(s, str) and s not in DELIMITERS and s != 'lambda'

############
## Parser ##
############
def read_expr(src):
    token = src.remove_front()
    if token is None:
        raise SyntaxError('Incomplete expression')
    elif is_literal(token):
        return read_call_expr(src, Literal(token))
    elif is_name(token):
        return read_call_expr(src, Name(token))
    elif token == 'lambda':
        params = read_comma_separated(src, read_param)
        src.expect(':')
        body = read_expr(src)
        return LambdaExpr(params, body)
    elif token == '(':
        inner_expr = read_expr(src)
        src.expect(')')
        return read_call_expr(src, inner_expr)
    else:
        raise SyntaxError("'{}' is not the start of an expression".format(token))

def read_comma_separated(src, reader):
    if src.current() in (':', ')'):
        return []
    else:
        s = [reader(src)]
        while src.current() == ',':
            src.remove_front()
            s.append(reader(src))
        return s

def read_call_expr(src, operator):
    while src.current() == '(':
        src.remove_front()
        operands = read_comma_separated(src, read_expr)
        src.expect(')')
        operator = CallExpr(operator, operands)
    return operator

def read_param(src):
    token = src.remove_front()
    if is_name(token):
        return token
    else:
        raise SyntaxError("Expected parameter name but got '{}'".format(token))

In [5]:
class PrimitiveFunction(Value):
    """A built-in function. For a full list of built-in functions, see
    `global_env` at the bottom of this file.

    The `operator` attribute is a Python function takes Python numbers and
    returns a Python number.
    """
    def __init__(self, operator):
        Value.__init__(self, operator)
        self.operator = operator

    def apply(self, arguments):
        for arg in arguments:
            if type(arg) != Number:
                raise TypeError("Invalid arguments {} to {}".format(
                    comma_separated(arguments), self))
        return Number(self.operator(*[arg.value for arg in arguments]))

    def __str__(self):
        return '<primitive function {}>'.format(self.operator.__name__)

# The environment that the REPL evaluates expressions in.
global_env = {
    'abs': PrimitiveFunction(operator.abs),
    'add': PrimitiveFunction(operator.add),
    'float': PrimitiveFunction(float),
    'floordiv': PrimitiveFunction(operator.floordiv),
    'int': PrimitiveFunction(int),
    'max': PrimitiveFunction(max),
    'min': PrimitiveFunction(min),
    'mod': PrimitiveFunction(operator.mod),
    'mul': PrimitiveFunction(operator.mul),
    'pow': PrimitiveFunction(pow),
    'sub': PrimitiveFunction(operator.sub),
    'truediv': PrimitiveFunction(operator.truediv),
}

In [12]:
class LambdaFunction(Value):
    """A lambda function. Lambda functions are created in the LambdaExpr.eval
    method. A lambda function is a lambda expression that knows the
    environment in which it was evaluated in.

    The `parameters` attribute is a list of variable names (a list of strings).
    The `body` attribute is an instance of `Expr`, the body of the function.
    The `parent` attribute is an environment, a dictionary with variable names
        (strings) as keys and instances of the class Value as values.
    """
    def __init__(self, parameters, body, parent):
        Value.__init__(self, parameters, body, parent)
        self.parameters = parameters
        self.body = body
        self.parent = parent

    def apply(self, arguments):
        """
        >>> from reader import read
        >>> add_lambda = read('lambda x, y: add(x, y)').eval(global_env)
        >>> add_lambda.apply([Number(1), Number(2)])
        Number(3)
        >>> add_lambda.apply([Number(3), Number(4)])
        Number(7)
        >>> sub_lambda = read('lambda add: sub(10, add)').eval(global_env)
        >>> sub_lambda.apply([Number(8)])
        Number(2)
        >>> add_lambda.apply([Number(8), Number(10)]) # Make sure you made a copy of env
        Number(18)
        >>> read('(lambda x: lambda y: add(x, y))(3)(4)').eval(global_env)
        Number(7)
        >>> read('(lambda x: x(x))(lambda y: 4)').eval(global_env)
        Number(4)
        """
        if len(self.parameters) != len(arguments):
            raise TypeError("Cannot match parameters {} to arguments {}".format(
                comma_separated(self.parameters), comma_separated(arguments)))
        "*** YOUR CODE HERE ***"
        print(self.parameters)        # ['x', 'y']
        print(self.body)              # add(x, y)
        # print(self.parent)    
        test = zip(self.parameters, arguments)
        vals = dict(test)
        
        operands = [operand.eval(vals) for operand in self.body.operands]
        operator = self.body.operator.eval(self.parent)
        return operator.apply(operands)


    def __str__(self):
        definition = LambdaExpr(self.parameters, self.body)
        return '<function {}>'.format(definition)


In [13]:
add_lambda = read('lambda x, y: add(x, y)').eval(global_env)
add_lambda.apply([Number(1), Number(2)])

['x', 'y']
add(x, y)


Number(3)

In [18]:
read('(lambda x: lambda y: add(x, y))(3)(4)').eval(global_env)

['x']
lambda y: add(x, y)


AttributeError: 'LambdaExpr' object has no attribute 'operands'

In [1]:
# Porject: Scheme

In [5]:
buf = Buffer(iter([["(+ 1 ", "(23 4)) ("]]))

In [8]:
buf.remove_front()

In [11]:
cd C:\Users\sisun\OneDrive\Documents\CS61A\week11\scheme

C:\Users\sisun\OneDrive\Documents\CS61A\week11\scheme


In [12]:
from scheme_reader import *

In [67]:
tokens = tokenize_lines(["(+ 1 ", "(23 4)) ("])

In [68]:
src = Buffer(tokens)

In [69]:
src.remove_front()

'('

In [70]:
scheme_read(src)

'+'

In [71]:
src.current()

1

In [72]:
scheme_read(Buffer(tokenize_lines(['(18 6)'])))

In [73]:
read_tail(Buffer(tokenize_lines(['1 2 3)'])))

In [1]:
ls

 Volume in drive C is Windows
 Volume Serial Number is 2223-DF9B

 Directory of C:\Users\sisun\OneDrive\Documents\CS61A\week11

01/08/2023  10:04 PM    <DIR>          .
01/04/2023  09:36 PM    <DIR>          ..
01/08/2023  05:36 PM    <DIR>          .ipynb_checkpoints
01/04/2023  09:37 PM    <DIR>          hw09
01/08/2023  05:44 PM    <DIR>          lab10
01/09/2023  06:14 PM    <DIR>          scheme
01/08/2023  10:04 PM            32,128 week11_notes.ipynb
               1 File(s)         32,128 bytes
               6 Dir(s)  506,512,646,144 bytes free


In [4]:
cd C:\Users\sisun\OneDrive\Documents\CS61A\week11\scheme

C:\Users\sisun\OneDrive\Documents\CS61A\week11\scheme


In [5]:
ls

 Volume in drive C is Windows
 Volume Serial Number is 2223-DF9B

 Directory of C:\Users\sisun\OneDrive\Documents\CS61A\week11\scheme

01/09/2023  11:31 PM    <DIR>          .
01/09/2023  11:29 PM    <DIR>          ..
01/09/2023  11:31 PM               498 .ok_history
01/09/2023  11:20 PM                 6 .ok_messages
01/09/2023  11:20 PM               227 .ok_storage.bak
01/09/2023  11:20 PM             3,142 .ok_storage.dat
01/09/2023  11:20 PM               227 .ok_storage.dir
01/09/2023  11:31 PM    <DIR>          __pycache__
01/04/2023  09:37 PM             4,224 buffer.py
01/04/2023  09:37 PM    <DIR>          images
01/04/2023  09:37 PM               432 mytests.rst
01/08/2023  10:11 PM         2,406,118 ok
01/04/2023  09:37 PM             1,092 proj04.ok
01/04/2023  09:37 PM             1,836 questions.scm
01/09/2023  11:22 PM            22,677 scheme.py
01/04/2023  09:37 PM            14,563 scheme_builtins.py
01/09/2023  11:36 PM             6,753 scheme_reader.py
01/04/2023

In [6]:
from scheme_reader import *
from scheme import *

In [5]:
expr = read_line('(+ 2 2)')
scheme_eval(expr, create_global_frame())

4

In [9]:
expr = read_line('(+ 2 2)')
first, rest = expr.first, expr.second
rest

Pair(2, Pair(2, nil))

In [6]:
expr = read_line('(+ (+ 2 2) (+ 1 3) (* 1 4))')
expr

Pair('+', Pair(Pair('+', Pair(2, Pair(2, nil))), Pair(Pair('+', Pair(1, Pair(3, nil))), Pair(Pair('*', Pair(1, Pair(4, nil))), nil))))

In [15]:
expr = read_line('(define size 2)')
expressions = expr.second
expressions

Pair('size', Pair(2, nil))

In [16]:
check_form(expressions, 2) 

In [19]:
target = expressions.first
if scheme_symbolp(target):
    print('yes')
    check_form(expressions, 2, 2)

yes


In [21]:
env = create_global_frame()
env

<Global Frame>

In [22]:
env[target] = expressions.second.first

TypeError: 'Frame' object does not support item assignment

In [35]:
expr = read_line(' ''hello')
# expr = read_line(expr)
# expr.first
expr

'hello'

In [29]:
scheme_eval(expr, env)

SchemeError: unknown identifier: hello

In [1]:
from scheme_reader import *
from scheme import *

ModuleNotFoundError: No module named 'scheme_reader'

In [47]:
quotes = {"'":  'quote',
          '`':  'quasiquote',
          ',':  'unquote'}

In [88]:
def scheme_read(src):
    """Read the next expression from SRC, a Buffer of tokens.
    """
    if src.current() is None:
        raise EOFError
    # print(src)
    val = src.remove_front() # Get the first token
    print(val)
    if val == 'nil':
        # BEGIN PROBLEM 1
        return nil
        # END PROBLEM 1
    elif val == '(':
        # BEGIN PROBLEM 1
        print('(')
        return read_tail(src)
        # END PROBLEM 1
    elif val in quotes:
        # BEGIN PROBLEM 7
        print(src.current())
        #test = read_tail(src)
        #print(test)
        src.remove_front()
        return Pair('quote', Pair(src.current(), nil))
        # END PROBLEM 7
    elif val not in DELIMITERS:
        return val
    else:
        raise SyntaxError('unexpected token: {0}'.format(val))

In [84]:
src = Buffer(tokenize_lines([" ''hello "]))
val = src.remove_front()
print(val)
src.remove_front()
print(src.current())

'
hello


In [89]:
scheme_read(Buffer(tokenize_lines([" ''hello "])))

'
'


Pair('quote', Pair('hello', nil))

In [8]:
 read_tail(Buffer(tokenize_lines(['2 (3 4))'])))

Pair(2, Pair(Pair(3, Pair(4, nil)), nil))